In [1]:
import json
import os
from sqlalchemy import create_engine #데이터 베이스와 연결하기 위한 라이브러리
from sqlalchemy import MetaData #메타데이터를 받아오기 위한 라이브러리
from sqlalchemy.engine import reflection #데이터 베이스 내의 정보를 가져오기 위한 라이브러리
from faker import Faker #더미데이터만들기 위해 사용

In [2]:
from tools.temp_tool import get_connectData #연결 정보를 가져오는 함수 import

In [3]:
connectData = get_connectData('setting/config.json')

In [4]:
def connect_DB(connectData : dict) :
    connection_string = f'{connectData['mysql-login']['db_type']}://{connectData['mysql-login']['user_name']}:{connectData['mysql-login']['user_pw']}@{connectData['mysql-login']['db_server']}:{connectData['mysql-login']['db_port']}/{connectData['mysql-login']['db_name']}?charset={connectData['mysql-login']['charset']}'
    engine = create_engine(connection_string, echo=False)
    #메타데이터 가져오기
    metadata = MetaData()
    # 데이터베이스에서 테이블 정보 반영
    metadata.reflect(bind=engine)
    # 터미널 내용 삭제
    # os.system('clear')
    # 데이터베이스 인스펙터 객체 생성
    inspector = reflection.Inspector.from_engine(engine)
    return engine, metadata, inspector

In [5]:
engine, metadata, inspector = connect_DB(connectData)

/var/folders/_0/sglhm_1s48z2d21krx9htrb40000gn/T/ipykernel_7516/313679101.py:11: SADeprecationWarning: The from_engine() method on Inspector is deprecated and will be removed in a future release.  Please use the sqlalchemy.inspect() function on an Engine or Connection in order to acquire an Inspector. (deprecated since: 1.4)
  inspector = reflection.Inspector.from_engine(engine)


In [6]:
def get_table_lists(inspector, table_name:str) :
    #테이블 이름 가져오기, 리스트 형식
    return inspector.get_table_names(table_name)

In [7]:
tables = get_table_lists(inspector,'airportdb')

In [32]:
def get_tables_detail(tables, inspector): 
    table_detail = dict()
    #정보 및 제약조건 가져오기
    for table in tables :
        if table not in table_detail.keys() : 
            table_detail.update({table : {}})
        columns = inspector.get_columns(table)
        temp = []
        # temp['constrained_columns'] = inspector.get_pk_constraint(table)['constrained_columns']
        for column in columns:
            # Unique 제약 확인
            unique_constraints = inspector.get_unique_constraints(table)
            if any(column['name'] in uc['column_names'] for uc in unique_constraints) :
                column['unique'] = True
            else :
                column['unique'] = False
            temp.append(column)
        #PK 제약조건 가져오기
        temp.append({'table_pk_details' :inspector.get_pk_constraint(table)['constrained_columns']})
        table_detail.update({table : {'details' : temp}})
        
    # del temp
        del temp
    return table_detail

In [10]:
def get_view_list(inspector) :
    return inspector.get_view_names()

In [13]:
view_list = get_view_list(inspector)

In [25]:
def get_view_detail(view_list, inspector) : 
    view_detail = dict()
    #정보 및 제약조건 가져오기
    for view in view_list :
        if view not in view_detail.keys() : 
            view_detail.update({view : {}})
        columns = inspector.get_columns(view)
        temp = []
        # temp['constrained_columns'] = inspector.get_pk_constraint(table)['constrained_columns']
        for column in columns:
            # Unique 제약 확인
            unique_constraints = inspector.get_unique_constraints(view)
            if any(column['name'] in uc['column_names'] for uc in unique_constraints) :
                column['unique'] = True
            else :
                column['unique'] = False
            temp.append(column)
        #PK 제약조건 가져오기
        temp.append({'table_pk_details' :inspector.get_pk_constraint(view)['constrained_columns']})
        view_detail.update({view : {'details' : temp}})
        
    # del temp
        del temp
    return view_detail

In [27]:
table = tables[0]

'airline'

In [30]:
def get_table_detail(table, inspector) :
    table_detail = dict()
    #정보 및 제약조건 가져오기

    if table not in table_detail.keys() : 
        table_detail.update({table : {}})
    columns = inspector.get_columns(table)
    temp = []
    # temp['constrained_columns'] = inspector.get_pk_constraint(table)['constrained_columns']
    for column in columns:
        # Unique 제약 확인
        unique_constraints = inspector.get_unique_constraints(table)
        if any(column['name'] in uc['column_names'] for uc in unique_constraints) :
            column['unique'] = True
        else :
            column['unique'] = False
        temp.append(column)
    #PK 제약조건 가져오기
    temp.append({'table_pk_details' :inspector.get_pk_constraint(table)['constrained_columns']})
    table_detail.update({table : {'details' : temp}})
        
    del temp
    return table_detail

In [33]:
tables_detail = get_tables_detail(tables, inspector)

In [34]:
tables_detail

{'airline': {'details': [{'name': 'airline_id',
    'type': SMALLINT(),
    'default': None,
    'comment': None,
    'nullable': False,
    'autoincrement': True,
    'unique': False},
   {'name': 'iata',
    'type': CHAR(charset='utf8mb4', collation='utf8mb4_unicode_ci', length=2),
    'default': None,
    'comment': None,
    'nullable': False,
    'unique': True},
   {'name': 'airlinename',
    'type': VARCHAR(charset='utf8mb4', collation='utf8mb4_unicode_ci', length=30),
    'default': None,
    'comment': None,
    'nullable': True,
    'unique': False},
   {'name': 'base_airport',
    'type': SMALLINT(),
    'default': None,
    'comment': None,
    'nullable': False,
    'autoincrement': False,
    'unique': False},
   {'table_pk_details': ['airline_id']}]},
 'airplane': {'details': [{'name': 'airplane_id',
    'type': INTEGER(),
    'default': None,
    'comment': None,
    'nullable': False,
    'autoincrement': True,
    'unique': False},
   {'name': 'capacity',
    'type': 